This function computes the interpolant for the Lagrangian vorticity barrier equation:

\begin{equation}
    \mathbf{x}_0'= \dfrac{\nu}{t_1-t_0} \mathbf{J} \nabla \delta \hat{\omega} (\mathbf{x}_0, t_0,t_1), \label{eq: vorticity} \tag{1}
\end{equation}

where $ \nu $ is the viscosity, $ \mathbf{J} = \begin{pmatrix} 0 && 1 \\ -1 && 0 \end{pmatrix} $ and $ \delta \hat{\omega} (\mathbf{x}_0, t_0,t_1) = \hat{\omega} (\mathbf{F}_{t_0}^{t_1}(\mathbf{x}_0), t_1) - \hat{\omega}(\mathbf{x}_0, t_0) $.
 
For simplicity we can normalize equation $ \ref{eq: vorticity} $ by $ \nu $ and thus simply omit them as they play no role when exploring the structures of the field $ \mathbf{x'} $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X_domain | array (Ny, Nx)| X-meshgrid of domain|
| Y_domain | array (Ny, Nx)| Y-meshgrid of domain|
| X | array (NY, NX)| X-meshgrid of data with NY=NX=1024 |
| Y | array (NY, NX)| Y-meshgrid of data with NY=NX=1024 |
| omega | array (NY, NX, NT)| vorticity over meshgrid and time |
| Fmap | array (Ny, Nx, N) | $ \mathbf{F}_{t_0}^{t}(\mathbf{x}_0) $, with $ t \in [t_0, t_N] $  |
| time | array (N, ) | $ t \in [t_0, t_N] $ |
| time_data | array | time of the velocity data |
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| Interpolant_Jgrad_delta_omega | list (2,) | Interpolant[0]: Interpolant for x-component of $ \mathbf{x_0}' $ <br /> Interpolant[1]: Interpolant for y-component of $ \mathbf{x_0}' $ |

In [ ]:
# import sys and os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

In [ ]:
# Import package for parallel computing
from joblib import Parallel, delayed

# Import package for progress bar
from tqdm.notebook import tqdm

# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

# Import package for computing trajectories/velocity along trajectories
from ipynb.fs.defs.integration_dFdt import integration_dFdt

# Import numpy 
import numpy as np

In [3]:
def LagrangianActiveVorticity(X_domain, Y_domain, X, Y, omega, Fmap, time, time_data, aux_grid):
    '''
    Compute interpolant of right-hand-side of Lagrangian vorticity barrier equation.
    
    Parameters:
        X_domain:  array(Ny, Nx), X-meshgrid of domain
        Y_domain:  array(Ny, Nx), Y-meshgrid of domain
        X:         array(NY, NX), X-meshgrid of data domain
        Y:         array(NY, NX), Y-meshgrid of data domain
        omega:     array(NY, NX, NT), 3D meshgrid of vorticity over spatial and temporal data domain
        Fmap:      array(Nt, 2, Ny*Nx), Flow map of trajectories
        time:      array(Nt, ), time
        time_data: array(1,NT), time data
        aux_grid:  list(2,), specifies spacing of x/y auxiliary grid components
        
    Returns:
        Interpolant_Jgrad_delta_omega: list(2,) of Interpolant-objects for right-hand-side of Lagrangian vorticity.
        Interpolant_Jgrad_delta_omega[0] --> Interpolant for x-component
        Interpolant_Jgrad_delta_omega[1] --> Interpolant for y-component
    '''
    
    # auxiliary grid
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    # iterate over all trajectories and compute omega along them
    
    # compute interpolant for omega over meshgrid X, Y over time
    Interpolant_omega = []
    
    # vorticity at t_0
    Interpolant_omega.append(RBS(Y[:,0], X[0,:], omega[:,:,0]))

    dt_data = time_data[0,1]-time_data[0,0]
    k = np.searchsorted(time_data[0,:], time[-1])
    
    if k >= time_data.shape[1]:
    
        Omega = (time_data[0,k+1]-time[-1])/dt_data*omega[:,:,k] + (time[-1]-time_data[0,k])/dt_data*omega[:,:,k+1]
        
    else:
        
        Omega = (time_data[0,k+1]-time[-1])/dt_data*omega[:,:,k] + (time[-1]-time_data[0,k])/dt_data*omega[:,:,k+1]
    
    # vorticity at t = t1 (=time[-1])
    Interpolant_omega.append(RBS(Y[:,0], X[0,:], Omega))
    
    # evaluate delta_omega at at time 't_1' 
    delta_omega = Interpolant_omega[-1](Fmap[-1,1,:].ravel(), Fmap[-1,0,:].ravel(), grid = False)-Interpolant_omega[0](Fmap[0,1,:].ravel(), Fmap[0,0,:].ravel(), grid = False)
    
    # Interpolant for delta_omega
    Interpolant_delta_omega = RBS(Y_domain[:,0], X_domain[0,:], delta_omega.reshape((X_domain.shape[0], Y_domain.shape[1])))
    
    Jgrad_delta_omega = np.zeros((X_domain.shape[0], Y_domain.shape[1], 2))*np.nan
        
    # compute x'=J grad(delta_omega), with J = [[0, 1], [-1, 0]]
    for i in range(X_domain.shape[0]):
        
        for j in range(Y_domain.shape[1]):
            
            # apply periodic boundary conditions to point 'x'
            x = np.array([X_domain[i, j], Y_domain[i, j]])
            
            # evaluate grad(omega) at 'x' using auxiliary grid
            xR = (x[0] + rho_x)%(X[0,-1]-X[0,0])
            OmegaR = Interpolant_delta_omega(x[1], xR)[0][0]
            
            xL = (x[0] - rho_x)%(X[0,-1]-X[0,0])
            OmegaL = Interpolant_delta_omega(x[1], xL)[0][0]
            
            xU = (x[1] + rho_y)%(Y[-1,0]-Y[0,0])
            OmegaU = Interpolant_delta_omega(xU, x[0])[0][0]
            
            xD = (x[1] - rho_y)%(Y[-1,0]-Y[0,0])
            OmegaD = Interpolant_delta_omega(xD, x[0])[0][0]
            
            Jgrad_delta_omega[i, j, 0] = -(OmegaU - OmegaD)/(2*rho_y)
            Jgrad_delta_omega[i, j, 1] = (OmegaR - OmegaL)/(2*rho_x)
    
    Jgrad_delta_omega = Jgrad_delta_omega/abs(time[-1]-time[0])
    
    Interpolant_Jgrad_delta_omega = []
    Interpolant_Jgrad_delta_omega.append(RBS(Y_domain[:,0], X_domain[0,:], Jgrad_delta_omega[:,:,0]))
    Interpolant_Jgrad_delta_omega.append(RBS(Y_domain[:,0], X_domain[0,:], Jgrad_delta_omega[:,:,1]))
    
    return Interpolant_Jgrad_delta_omega